In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Input
import keras.backend as K
import tensorflow as tf

import numpy as np

from matplotlib import pyplot as plt
plt.style.use('seaborn-white')
from matplotlib import cm

from utils import iter_or_rep
from GaussianAnsatz.dnn import DNN
from GaussianAnsatz.ifn import IFN, GaussianAnsatz
from GaussianAnsatz.utils import build_gaussianAnsatz_DNN